In [1]:
import pandas as pd
import numpy as np

import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor

import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

from gensim.models import Word2Vec

import spacy
import pandas as pd
import re

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")

import pandas as pd
import numpy as np
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, r2_score
import joblib

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor


In [2]:
path = 'clean_data_with_same_units.csv'
df = pd.read_csv(path, index_col=0)

In [3]:
# Store a copy of the original numeric nutrition data (before modifying df)
df_original = df.copy()

In [4]:
# Load spaCy NLP model
nlp = spacy.load("en_core_web_sm")

def parse_food_name(food_name):
    """Extract meaningful components from food name using NLP."""
    doc = nlp(food_name.lower())
    return [token.lemma_ for token in doc if not token.is_stop]  # Lemmatization & stop-word removal

# Apply parsing
df["parsed_food"] = df["food"].apply(parse_food_name)

# Extract unique components
all_components = list(set(sum(df["parsed_food"].tolist(), [])))
print("Extracted Components:", all_components[:10])  # Check some parsed components


Extracted Components: ['okara', 'flower', 'tannier', 'alpro', 'yardlong', 'pectin', 'purslane', 'hot', 'scup', 'woca']


In [5]:
# Create a binary matrix (food × components)
component_df = pd.DataFrame(0, index=df.index, columns=all_components)

# Populate the one-hot matrix
for i, components in enumerate(df["parsed_food"]):
    for component in set(components):  # Use set to avoid duplicate labels
        if component in component_df.columns:  # Ensure valid column
            component_df.at[i, component] = 1

# Merge back into original dataframe
df = pd.concat([df, component_df], axis=1)
print(df.head())  # Check structure


                               food  Caloric Value    Fat  Saturated Fats  \
0                      cream cheese             51   5000            2900   
1                 neufchatel cheese            215  19400           10900   
2  requeijao cremoso light catupiry             49   3600            2300   
3                    ricotta cheese             30   2000            1300   
4              cream cheese low fat             30   2300            1400   

   Monounsaturated Fats  Polyunsaturated Fats  Carbohydrates  Sugars  Protein  \
0                  1300                   200            800     500      900   
1                  4900                   800           3100    2700     7800   
2                   900                     0            900    3400      800   
3                   500                     2           1500      91     1500   
4                   600                    42           1200     900     1200   

   Dietary Fiber  ...  scallop  sesbania  chantere

In [6]:
# Only process parsed components separately
component_df = component_df.copy()  # Ensure it's separate from df

In [7]:
# Verify numeric columns are coming from the original dataset
all_nutrition_columns = df_original.select_dtypes(include=[np.number]).columns.tolist()
all_nutrition_columns.remove("Nutrition Density")  # Exclude target variable

In [8]:
all_nutrition_columns

['Caloric Value',
 'Fat',
 'Saturated Fats',
 'Monounsaturated Fats',
 'Polyunsaturated Fats',
 'Carbohydrates',
 'Sugars',
 'Protein',
 'Dietary Fiber',
 'Cholesterol',
 'Sodium',
 'Water',
 'Vitamin A',
 'Vitamin B1',
 'Vitamin B11',
 'Vitamin B12',
 'Vitamin B2',
 'Vitamin B3',
 'Vitamin B5',
 'Vitamin B6',
 'Vitamin C',
 'Vitamin D',
 'Vitamin E',
 'Vitamin K',
 'Calcium',
 'Copper',
 'Iron',
 'Magnesium',
 'Manganese',
 'Phosphorus',
 'Potassium',
 'Selenium',
 'Zinc']

In [9]:
# Convert each numeric column in the original dataset
for col in all_nutrition_columns:
    df_original[col] = pd.to_numeric(df_original[col], errors='coerce')  # Convert to float
    df_original[col] = df_original[col].fillna(0)  # Fill NaN with 0

# Check if everything is float64
print(df_original[all_nutrition_columns].dtypes)

Caloric Value             int64
Fat                       int64
Saturated Fats            int64
Monounsaturated Fats      int64
Polyunsaturated Fats      int64
Carbohydrates             int64
Sugars                    int64
Protein                   int64
Dietary Fiber             int64
Cholesterol             float64
Sodium                  float64
Water                     int64
Vitamin A               float64
Vitamin B1              float64
Vitamin B11             float64
Vitamin B12             float64
Vitamin B2              float64
Vitamin B3              float64
Vitamin B5              float64
Vitamin B6              float64
Vitamin C               float64
Vitamin D               float64
Vitamin E               float64
Vitamin K               float64
Calcium                 float64
Copper                  float64
Iron                    float64
Magnesium               float64
Manganese               float64
Phosphorus              float64
Potassium               float64
Selenium

In [10]:
import numpy as np

# Prepare matrices
X = component_df.values  # One-hot matrix (food × components)
Y = df_original[all_nutrition_columns].values.astype(np.float64)  # Nutrient matrix (food × nutrients)

# Solve least squares problem
W, residuals, rank, s = np.linalg.lstsq(X, Y, rcond=None)

# Convert to DataFrame
component_nutrition_all = pd.DataFrame(W, index=component_df.columns, columns=all_nutrition_columns)

# Display results
print(" Independent Nutrient Contributions (All Nutrients):")
print(component_nutrition_all.head(10))


 Independent Nutrient Contributions (All Nutrients):
          Caloric Value          Fat  Saturated Fats  Monounsaturated Fats  \
okara         93.000000  2100.000000      200.000000          4.000000e+02   
flower      -126.976522 -9418.498146    -2623.098926         -3.446721e+03   
tannier       66.000000   250.000000       50.000000         -4.670255e-10   
alpro          5.250000   150.000000       25.000000          6.750000e+02   
yardlong      98.093675 -4240.168435    -1111.409707         -1.603543e+03   
pectin       163.000000   200.000000       52.000000          9.300000e+01   
purslane    -113.122189 -6371.147974    -2271.975645         -2.712757e+03   
hot           59.169751  2546.233292      850.984764          1.788923e+03   
scup          -7.122189 -3421.147974    -1721.975645         -2.262757e+03   
woca         -76.266323 -9451.543051     -641.669251         -2.192878e+03   

          Polyunsaturated Fats  Carbohydrates        Sugars       Protein  \
okara      

In [11]:
# Create a version where negative values are kept (for predictions)
component_nutrition_prediction = component_nutrition_all.copy()

# Create a version where negative values are replaced with 0 (for user display)
component_nutrition_display = component_nutrition_all.clip(lower=0)

# Display the first 10 rows of each
print(" Independent Nutrient Contributions (For Predictions - Allow Negatives):")
print(component_nutrition_prediction.head(10))

print("\n Independent Nutrient Contributions (For User Display - No Negatives):")
print(component_nutrition_display.head(10))


 Independent Nutrient Contributions (For Predictions - Allow Negatives):
          Caloric Value          Fat  Saturated Fats  Monounsaturated Fats  \
okara         93.000000  2100.000000      200.000000          4.000000e+02   
flower      -126.976522 -9418.498146    -2623.098926         -3.446721e+03   
tannier       66.000000   250.000000       50.000000         -4.670255e-10   
alpro          5.250000   150.000000       25.000000          6.750000e+02   
yardlong      98.093675 -4240.168435    -1111.409707         -1.603543e+03   
pectin       163.000000   200.000000       52.000000          9.300000e+01   
purslane    -113.122189 -6371.147974    -2271.975645         -2.712757e+03   
hot           59.169751  2546.233292      850.984764          1.788923e+03   
scup          -7.122189 -3421.147974    -1721.975645         -2.262757e+03   
woca         -76.266323 -9451.543051     -641.669251         -2.192878e+03   

          Polyunsaturated Fats  Carbohydrates        Sugars       Pr

component_nutrition_prediction → Keeps negative values (for further calculations)

component_nutrition_display → Replaces negatives with 0 (for user queries)

In [26]:
ridge_selected_features = ['Caloric Value', 'Sodium', 'Vitamin A', 'Vitamin B12', 'Vitamin B2', 'Vitamin C', 'Calcium', 'Iron', 'Protein', 'Fat', 'Carbohydrates']

#### Model 1

In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Add macro nutrients to Ridge-selected features
improved_features = ridge_selected_features + ["Fat", "Protein", "Carbohydrates"]

# Prepare training dataset
X_train_list = []
y_train_list = []

for food, row in df_original.iterrows():
    components = parse_food_name(row["food"])  
    known_components = [c for c in components if c in component_nutrition_prediction.index]

    if not known_components:
        continue  

    # Sum contributions of known components
    summed_nutrition = component_nutrition_prediction.loc[known_components].sum()

    # Extract only the improved feature set
    X_train_list.append(summed_nutrition[improved_features].values)
    y_train_list.append(row["Nutrition Density"])  

# Convert lists to NumPy arrays
X_train = np.array(X_train_list)
y_train = np.array(y_train_list)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Apply Standard Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [33]:
# Train a Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Make predictions
y_pred = rf_model.predict(X_test_scaled)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Improved Model Evaluation Results:")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R² Score: {r2:.4f}")


Improved Model Evaluation Results:
Mean Absolute Error (MAE): 42.2301
Mean Squared Error (MSE): 6318.4725
R² Score: 0.7411


In [74]:
def predict_nutrition_density(food_name):
    """
    Predicts Nutrition Density for a given food name using the trained Random Forest model.
    If the food name is not recognized, it informs the user that it may be updated in the future.
    """
    components = parse_food_name(food_name)
    known_components = [c for c in components if c in component_nutrition_prediction.index]

    if not known_components:
        return f"⚠️ '{food_name}' is not available in our database yet. Please wait for future updates."

    # Sum up nutrition contributions
    summed_nutrition = component_nutrition_prediction.loc[known_components].sum()

    # Select only the improved feature set
    X_input = summed_nutrition[improved_features].values.reshape(1, -1)

    # Apply scaling
    X_input_scaled = scaler.transform(X_input)

    # Predict Nutrition Density
    predicted_density = rf_model.predict(X_input_scaled)[0]

    return f"Predicted Nutrition Density for '{food_name}': {predicted_density:.4f}"


In [76]:
# Prediction
food_to_predict = "strawberry"
print(predict_nutrition_density(food_to_predict))

food_to_predict = "unico"  # Example of a nonexistent food
print(predict_nutrition_density(food_to_predict))

Predicted Nutrition Density for 'strawberry': 15.0184
⚠️ 'unico' is not available in our database yet. Please wait for future updates.


In [ ]:
import pickle
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# Assuming X_train and y_train exist in your dataset
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Save the trained model
with open("/Users/sarahxia/Desktop/ADSP 31017 ML/Project/random_forest_model.pkl", "wb") as f:
    pickle.dump(rf_model, f)

# Save the scaler
with open("/Users/sarahxia/Desktop/ADSP 31017 ML/Project/scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

print("✅ Model and Scaler saved successfully!")
